In [ ]:
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
main_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
main_train.head()

In [ ]:
y_data = main_train['Pawpularity']
main_train.drop('Pawpularity',axis=1,inplace=True)

In [ ]:
main_train.columns

In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Model


In [ ]:
base_model = ResNet50(weights='imagenet',input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
main_train.columns

In [ ]:
def make_train_file(train,training_dir,model):
    x_data = []
    for rows in tqdm(range(len(train))):
      get_file_name = train.loc[rows]  
      img = cv2.imread(training_dir+'/'+get_file_name['Id']+'.jpg')
      stretch_near = cv2.resize(img, (224, 224),interpolation = cv2.INTER_AREA)
      features = model.predict(stretch_near.reshape(1,224,224,3))
      features = features.reshape(2048,)
      index_title = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
      for d in index_title:
          features = np.append(features, get_file_name[d]) 
      x_data.append(features)
    return x_data


In [ ]:
training_dir='../input/petfinder-pawpularity-score/train'
total_data = make_train_file(main_train,training_dir,model)

In [ ]:
print(total_data[0])

In [ ]:
import pickle as pkl

with open('./x_train_data.pkl','wb') as f:
  pkl.dump(np.array(total_data), f)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(total_data, y_data, test_size=0.33, random_state=11)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
regressor = GradientBoostingRegressor(random_state=0)
regressor.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import r2_score


In [ ]:
y_pred = regressor.predict(x_test)
score = r2_score(y_test, y_pred)
print(score)

In [ ]:
main_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
test_dir = '../input/petfinder-pawpularity-score/test'
test_data = make_train_file(main_test,test_dir,model)

In [ ]:
y_pred = regressor.predict(test_data)

In [ ]:
main_result = []
main_result.append(['id','Pawpularity'])
for r in range(len(main_test)):
    id_row = main_test.loc[r]
    main_result.append([id_row[0],y_pred[r]])

   
np.savetxt("submission.csv", 
           np.array(main_result),
           delimiter =", ",fmt ='% s') 